# Commercial Data Analysis

#### In this project, I worked with a dataset containing information about commercial businesses where each each record is a business location. 

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("comm") \
        .getOrCreate()

In [2]:
import os

In [3]:
path_to_file = 'part-00000-a159c41a-bc58-4476-9b78-c437667f9c2b-c000.json.gz'

### Read in the dataset and show the number of records

In [4]:
df = spark.read.json(path_to_file)

df.count()

154679

### Inspect the schema

In [5]:
df.printSchema()

root
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- coordinates: struct (nullable = true)
 |    |    |-- lat: double (nullable = true)
 |    |    |-- lon: double (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- county: string (nullable = true)
 |    |-- full_address: string (nullable = true)
 |    |-- highway_number: string (nullable = true)
 |    |-- is_headquarters: boolean (nullable = true)
 |    |-- is_parsed: boolean (nullable = true)
 |    |-- post_direction: string (nullable = true)
 |    |-- pre_direction: string (nullable = true)
 |    |-- secondary_number: string (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- street: string (nullable = true)
 |    |-- street_address: string (nullable = true)
 |    |-- street_number: string (nullable = true)
 |    |-- street_type: string (nullable = true)
 |    |-- type_of_address: string (nullable = true)
 |    |-- zip: string (nullable = true)
 |    |-- 

### Look at some data

In [15]:
df.select('*').show(5, True)

+--------------------+--------------------+--------------------+----------------+----+--------------------+--------------------+--------------------+
|             address|       business_tags|               hours|              id|menu|             reviews|                urls|             webpage|
+--------------------+--------------------+--------------------+----------------+----+--------------------+--------------------+--------------------+
|[Woodburn, [45.15...|                null|                null|000023995a540868|null|                  []|[woodburn.k12.or....|[Educational Tech...|
|[Hialeah, [25.884...|[[], [[has_atm, Y...|[, 1900,, 0830, 3...|0000821a1394916e|null|                null|[, [yelp.com],,, ...|                null|
|[Rochester, [43.1...|[[], [[accepts_cr...|[, 1700,, 0830, 3...|000136e65d50c3b7|null|[[New (to me) qui...|[usps.com, [yelp....|[Welcome | USPS G...|
|[West Palm Beach,...|                null|                null|00014329a70b9869|null|              

### Count the number of records where the city is Houston

In [10]:
df.filter(df.address.city=='Houston').count()

1668

### Count number of records where closing time on Friday is 7pm

In [11]:
df.filter(df.hours.friday_close=='1900').count()

3305

### Count the number of records where city is Houston and closing time on Friday is 7pm

In [12]:
df.filter((df.address.city=='Houston') & (df.hours.friday_close=='1900')).count()

42

### Price range is quoted in number of dollar signs.  Count the number of records with price range greater than or equal to three.

In [13]:
df.filter(df.menu.price_range >= 3).count()

115

### Count how many locations are HQ / are NOT HQ / are null?

In [14]:
df.groupBy('address.is_headquarters').count().show()

+---------------+-----+
|is_headquarters|count|
+---------------+-----+
|           null|87625|
|           true|  318|
|          false|66736|
+---------------+-----+



### Use Spark SQL to select the webpage title where the webpage url (accessed under `webpage.url`) is *Target.com*.  

### Show first result



In [16]:
df.registerTempTable('dff')
df2 = spark.sql('select webpage.title from dff where webpage.url == "Target.com"')

df2.show(1, False)

+-------------------------------+
|title                          |
+-------------------------------+
|Target : Expect More. Pay Less.|
+-------------------------------+
only showing top 1 row



### The reviews contains information such as the number of stars for each review (the *rating*).  
### The ratings are stored in an array (`reviews.stars`) for each business location. Return the top five most common rating arrays.



In [17]:
df.groupBy('reviews.stars').count().orderBy('count', ascending=False).show(5)

+------+-----+
| stars|count|
+------+-----+
|  null|74679|
|    []|42419|
|   [5]| 4258|
|    []| 3067|
|[5, 5]| 1610|
+------+-----+
only showing top 5 rows



### Filtering out null ratings and to compute the average rating for each business location.


### Create a new dataframe retaining two fields: `id`, `reviews.stars`


In [18]:
r = df.select('id', 'reviews.stars')

### Create a row for each rating  

In [19]:
from pyspark.sql.functions import explode
r = r.withColumn('rating', explode('stars'))

### Return a count of the number of ratings in this dataframe

In [20]:
r.count()

600082

In [21]:
r.show(5)

+----------------+------+------+
|              id| stars|rating|
+----------------+------+------+
|000136e65d50c3b7|[4, 4]|     4|
|000136e65d50c3b7|[4, 4]|     4|
|0003b7589a4e12a0|   [5]|     5|
|00045f958e4bb02a| [,,,]|  null|
|00045f958e4bb02a| [,,,]|  null|
+----------------+------+------+
only showing top 5 rows



### Drop rows where the rating is null, and return a count of the number of non-null ratings

In [22]:
rf = r.where(r.rating.isNotNull())
rf.count()

538241

### Compute the average rating, grouped by `id`. After the average is computed, sort by `id` in ascending order and show the top 10 records.  

In [23]:
rf.groupBy("id").agg({"rating":"avg"}).orderBy('id', ascending=True).show(10) 

+----------------+------------------+
|              id|       avg(rating)|
+----------------+------------------+
|000136e65d50c3b7|               4.0|
|0003b7589a4e12a0|               5.0|
|00059519f0dba1b4|3.3333333333333335|
|000a1df4c8e0ecd2|               4.6|
|000c7b7a30623083|               5.0|
|000c9ffc8b89af03|               3.0|
|000de20baa847ecc|1.6666666666666667|
|001064359d9f162f|               5.0|
|0010c9f495d87dd7|               3.0|
|0017774db5e6400a| 4.333333333333333|
+----------------+------------------+
only showing top 10 rows

